In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import mne
import numpy as np
from copy import deepcopy
import matplotlib.pyplot as plt
import sys; sys.path.insert(0, '../')
from esinet import util
from esinet import Simulation
from esinet import Net
from esinet.forward import create_forward_model, get_info
plot_params = dict(surface='white', hemi='both', verbose=0)

# Forward Model

In [2]:
info = get_info()
info['sfreq'] = 100
fwd = create_forward_model(info=info)

[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   3 out of   8 | elapsed:    1.4s remaining:    2.4s
[Parallel(n_jobs=8)]: Done   5 out of   8 | elapsed:    1.5s remaining:    0.9s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:    1.7s finished
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   3 out of   8 | elapsed:    0.0s remaining:    0.1s
[Parallel(n_jobs=8)]: Done   5 out of   8 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:    0.2s finished
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   3 out of   8 | elapsed:    0.2s remaining:    0.3s
[Parallel(n_jobs=8)]: Done   5 out of   8 | elapsed:    0.2s remaining:    0.1s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:    0.2s finished


# Simulate

In [3]:
n_samples = 10000
settings = dict(duration_of_trial=0.2, target_snr=(0.5, 10))

sim_lstm = Simulation(fwd, info, verbose=True, settings=settings).simulate(n_samples=n_samples)
sim_dense = util.convert_simulation_temporal_to_single(sim_lstm)

sim_lstm_test = Simulation(fwd, info, verbose=True, settings=settings).simulate(n_samples=1000)
sim_dense_test = util.convert_simulation_temporal_to_single(sim_lstm_test)

Simulate Source


  0%|          | 0/10000 [00:00<?, ?it/s]

Converting Source Data to mne.SourceEstimate object


  0%|          | 0/10000 [00:00<?, ?it/s]


Project sources to EEG...

Create EEG trials with noise...


  0%|          | 0/10000 [00:00<?, ?it/s]


Convert EEG matrices to a single instance of mne.Epochs...
Simulate Source


  0%|          | 0/1000 [00:00<?, ?it/s]

# Keras-Tuner

In [ ]:
import keras_tuner as kt
import tensorflow as tf
from esinet.net import build_nas_lstm

# tuner = kt.BayesianOptimization(
#     build_nas_lstm,
#     objective='val_loss',
#     max_trials=450, 
#     directory='keras-tuner',
#     project_name=f'holiday_run',
#     # max_model_size=int(2e6),  # Maximum 700k parameters per model
#     beta=2.6*2,  # exploration parameter
#     )    
tuner = kt.RandomSearch(
    build_nas_lstm,
    objective='val_loss',
    max_trials=450, 
    directory='keras-tuner',
    project_name=f'holiday_run',
    )
callbacks=[tf.keras.callbacks.EarlyStopping('val_loss', patience=15, min_delta=0.0001)]
net = Net(fwd)
x_train, y_train = net.prep_data(sim_lstm)
x_val, y_val = net.prep_data(sim_lstm_test)
tuner.search(x_train, y_train, epochs=150, validation_data=(x_val, y_val), 
    batch_size=8, callbacks=callbacks, shuffle=True)

best_model = tuner.get_best_models()[0]


Search: Running Trial #1

Hyperparameter    |Value             |Best Value So Far 
lstm_layers       |2                 |?                 
dense_layers      |2                 |?                 
activation_out    |sigmoid           |?                 
actvation_all     |elu               |?                 
learning_rate     |0.018376          |?                 
Clip Value        |0.31386           |?                 

Epoch 1/150
1250/1250 [==============================] - 16s 10ms/step - loss: 0.1794 - mse: 1.0194 - mae: 0.2337 - val_loss: 0.1814 - val_mse: 1.0186 - val_mae: 0.2357
Epoch 2/150
1250/1250 [==============================] - 13s 10ms/step - loss: 0.1790 - mse: 1.0184 - mae: 0.2316 - val_loss: 0.1814 - val_mse: 1.0186 - val_mae: 0.2357
Epoch 3/150
1250/1250 [==============================] - 13s 10ms/step - loss: 0.1790 - mse: 1.0184 - mae: 0.2317 - val_loss: 0.1813 - val_mse: 1.0185 - val_mae: 0.2363
Epoch 4/150
1250/1250 [==============================] - 13s 10ms/

Exception ignored in: <function IteratorResourceDeleter.__del__ at 0x000001A94757BDC0>
Traceback (most recent call last):
  File "C:\Users\lukas\virtualenvs\esienv\lib\site-packages\tensorflow\python\data\ops\iterator_ops.py", line 545, in __del__
    gen_dataset_ops.delete_iterator(
  File "C:\Users\lukas\virtualenvs\esienv\lib\site-packages\tensorflow\python\ops\gen_dataset_ops.py", line 1262, in delete_iterator
    _result = pywrap_tfe.TFE_Py_FastPathExecute(
KeyboardInterrupt: 


1250/1250 [==============================] - 12s 10ms/step - loss: 0.1780 - mse: 1.0078 - mae: 0.2357 - val_loss: 0.1805 - val_mse: 1.0097 - val_mae: 0.2389
Epoch 34/150
 681/1250 [===============>..............] - ETA: 5s - loss: 0.1782 - mse: 1.0077 - mae: 0.2359

# Visualize Keras Tuner results

In [1]:
import os
import json
import pandas as pd

base_path = 'keras-tuner/holiday_run'
folders = os.listdir(base_path)
folders = [folder for folder in folders if folder.startswith('trial_')]
hyperparams = []
scores = []
for folder in folders:
    # Load trial data
    with open(base_path + '/' + folder + '/trial.json') as json_file:
        data = json.load(json_file)
    try:
        # extract score
        scores.append( data['metrics']['metrics']['val_loss']['observations'][0]['value'][0] )
        # extract hyperparameters
        hyperparams.append( data['hyperparameters']['values'] )
    except:
        pass

dfs = []
for i, (score, hyper) in enumerate(zip(scores, hyperparams)):
    df = pd.DataFrame.from_dict(hyper, orient='index').T
    df['score'] = score
    dfs.append(df)

df = pd.concat(dfs).sort_values('score')
df


,lstm_layers,dense_layers,activation_out,learning_rate,Clip Value,lstm_units_l-0,dropout_lstm_l-0,activation_LSTM_l-0,dense_units_l-0,dropout_dense_l-0,...,dense_units_l-1,dropout_dense_l-1,activation_dense_l-1,dense_units_l-2,dropout_dense_l-2,activation_dense_l-2,lstm_units_l-2,dropout_lstm_l-2,activation_LSTM_l-2,score
0,0,0,sigmoid,0.009764,0.487891,53,0.504278,relu,232,0.746615,...,30,0.819406,sigmoid,130,0.433292,linear,177,0.265627,sigmoid,0.178093
0,0,1,linear,0.083285,0.174854,28,0.605605,swish,195,0.407561,...,58,0.746919,elu,208,0.226192,sigmoid,168,0.820055,sigmoid,0.178508
0,1,2,tanh,0.002344,0.395787,110,0.611545,linear,30,0.430766,...,213,0.821892,elu,60,0.152578,swish,64,0.289586,swish,0.178809
0,1,0,tanh,0.002429,0.743046,168,0.817094,linear,292,0.556742,...,94,0.178425,sigmoid,204,0.509599,elu,134,0.498293,linear,0.178881
0,1,1,linear,0.003782,0.875491,60,0.716961,relu,160,0.065718,...,187,0.623566,linear,164,0.006458,sigmoid,29,0.448039,swish,0.179182
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,3,1,sigmoid,0.062547,0.162525,186,0.632301,sigmoid,66,0.60209,...,45,0.829584,swish,290,0.733773,elu,147,0.715338,swish,NaN
0,1,2,linear,0.086794,0.382668,53,0.487244,elu,237,0.808963,...,118,0.474454,relu,213,0.070639,sigmoid,147,0.449501,relu,NaN
0,2,2,linear,0.085382,0.820841,130,0.196546,relu,92,0.123056,...,25,0.0,sigmoid,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,2,2,linear,0.078846,0.400367,177,0.419022,elu,180,0.371258,...,49,0.824206,linear,63,0.300334,swish,114,0.228853,linear,NaN


In [6]:
df.to_excel(r'C:\Users\lukas\Dokumente\projects\esinet\keras-tuner\holidayrun.xlsx')

# Build & train LSTM network

In [ ]:
epochs = 200
patience = 20
activation_funcion = 'relu'
loss = 'huber'
dropout = 0.2
# Train
# model_params = dict(activation_function=activation_funcion, n_dense_layers=2, 
#     n_dense_units=200)
# train_params = dict(epochs=epochs, patience=patience, tensorboard=True, 
#     dropout=dropout, loss=loss, optimizer='adam', return_history=True)
# net_dense = Net(fwd, **model_params)
# _, history_dense = net_dense.fit(sim_dense, **train_params)

# LSTM v2
model_params = dict(activation_function=activation_funcion, n_lstm_layers=2, 
    n_lstm_units=75, model_type='v2')
train_params = dict(epochs=epochs, patience=patience, tensorboard=True, 
    dropout=dropout, loss=loss, optimizer=None, return_history=True, 
    batch_size=8)

net_lstm_v2 = Net(fwd, **model_params)
_, history_lstm = net_lstm_v2.fit(sim_lstm, **train_params)

# LSTM v3
# model_params = dict(activation_function=activation_funcion, n_lstm_layers=2, 
#     n_lstm_units=75, model_type='v3')
# train_params = dict(epochs=epochs, patience=patience, tensorboard=True, 
#     dropout=dropout, loss=loss, optimizer=None, return_history=True, 
#     batch_size=8)

# net_lstm_v3 = Net(fwd, **model_params)
# _, history_lstm = net_lstm_v3.fit(sim_lstm, **train_params)




In [ ]:
%matplotlib qt
settings_eval = dict(duration_of_trial=0.2, target_snr=(0.5, 10))

# Simulate new data
sim_test = Simulation(fwd, info, settings=settings_eval).simulate(1)
idx = 0
# Predict sources
prediction_dense = net_dense.predict(sim_test)
prediction_lstm = net_lstm_v2.predict(sim_test)


# Plot True Source
brain = sim_test.source_data[idx].plot(**plot_params)
brain.add_text(0.1, 0.9, 'Ground Truth', 'title')

# Plot True EEG
evoked = sim_test.eeg_data[idx].average()
evoked.plot()
evoked.plot_topomap(title='Ground Truth')
evoked = util.get_eeg_from_source(sim_test.source_data[idx], fwd, info, tmin=0.)
evoked.plot_topomap(title='Ground Truth Noiseless')


# Plot predicted source Dense
brain = prediction_dense.plot(**plot_params)
brain.add_text(0.1, 0.9, 'Dense', 'title')
# Plot predicted EEG
evoked_esi = util.get_eeg_from_source(prediction_dense, fwd, info, tmin=0.)
evoked_esi.plot()
evoked_esi.plot_topomap(title='Dense')

# Plot predicted source LSTM
brain = prediction_lstm.plot(**plot_params)
brain.add_text(0.1, 0.9, 'LSTM', 'title')
# Plot predicted EEG
evoked_esi = util.get_eeg_from_source(prediction_lstm, fwd, info, tmin=0.)
evoked_esi.plot()
evoked_esi.plot_topomap(title='LSTM')

error_dense = ((prediction_dense.data - sim_test.source_data[idx].data)**2).flatten()
error_lstm = ((prediction_lstm.data - sim_test.source_data[idx].data)**2).flatten()

diff = error_dense - error_lstm
relative_better_predictions = np.sum(diff>0)/ len(diff)
title = f'{relative_better_predictions*100:.1f} % of samples were better with lstm'
print(title)

# Perform predictions on test set

In [ ]:
settings = dict(duration_of_trial=0.2, target_snr=(0.5, 10))
sim_lstm_test = Simulation(fwd, info, verbose=True, settings=settings).simulate(n_samples=20)
sim_dense_test = util.convert_simulation_temporal_to_single(sim_lstm_test)

In [ ]:
# predict
models = [net_dense, net_lstm_v2, net_lstm_v3]
model_names = [model.model.name for model in models]
predictions = [model.predict(sim_lstm_test) for model in models]

## Calc Mean Localization Error

In [ ]:
from esinet.evaluate import eval_mean_localization_error, eval_nmse, eval_auc, eval_mse
from scipy.spatial.distance import cdist
size = 30
pos = util.unpack_fwd(fwd)[2]
distance_matrix = cdist(pos, pos)

mean_localization_errors = []
aucs = []
nmses = []
mses = []
true_sources = np.stack([src.data for src in sim_lstm_test.source_data], axis=0)
true_sources = util.collapse(true_sources)
choice = np.random.choice(np.arange(true_sources.shape[0]), size=size, replace=False)
true_sources = true_sources[choice]
for prediction in predictions:
    predicted_sources = util.collapse(np.stack([src.data for src in prediction], axis=0))
    
    predicted_sources = predicted_sources[choice]
    
    print('mle calculation....')
    mean_localization_error = [eval_mean_localization_error(true_source, predicted_source, pos, distance_matrix=distance_matrix) for true_source, predicted_source in zip(true_sources, predicted_sources)]
    auc = [eval_auc(true_source, predicted_source, pos) for true_source, predicted_source in zip(true_sources, predicted_sources)]
    nmse = [eval_nmse(true_source, predicted_source) for true_source, predicted_source in zip(true_sources, predicted_sources)]
    mse = [eval_mse(true_source, predicted_source) for true_source, predicted_source in zip(true_sources, predicted_sources)]
    
    mean_localization_errors.append(mean_localization_error)
    aucs.append(auc)
    nmses.append(nmse)
    mses.append(mse)

aucs_far = [auc[1] for auc in aucs]
aucs_close = [auc[0] for auc in aucs]

In [ ]:
import seaborn as sns
%matplotlib qt

plt.figure()
sns.boxplot(data=nmses)
plt.title('Normalized Mean Squared Errors')

plt.figure()
sns.boxplot(data=mses)
plt.title('Mean Squared Errors')

plt.figure()
sns.boxplot(data=aucs_far)
plt.title('Far area under the curve')


plt.figure()
sns.boxplot(data=aucs_close)
plt.title('Close area under the curve')


plt.figure()
sns.boxplot(data=mean_localization_errors)
plt.title('Mean Localization Errors')
plt.tight_layout()

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(10,10))
ax.scatter(nmses[0], nmses[2], s=0.5)
ax.plot([0, 1], [0, 1], linewidth=2, color='black')
ax.set_xlim([-np.percentile(nmses[0], 5), np.percentile(nmses[0], 99)])
ax.set_ylim([-np.percentile(nmses[0], 5), np.percentile(nmses[0], 99)])
ax.set_xlabel('Dense')
ax.set_ylabel('LSTM v3')

In [ ]:
from scipy.stats import ttest_rel, wilcoxon
print('AUC_far:', ttest_rel(aucs_far[0], aucs_far[1]))
print('AUC_close:', ttest_rel(aucs_close[0], aucs_close[1]))
print('MLE:', wilcoxon(mean_localization_errors[0], mean_localization_errors[1]))
print('nMSE:', ttest_rel(nmses[0], nmses[1]))
print('MSE:', ttest_rel(mses[0], mses[1]))